# BDE Fördermitglieder – Partner-Links extrahieren

Dieses Notebook lädt die Übersichtsseite der BDE-Fördermitglieder, extrahiert alle Partner-Links (sowohl interne Profilseiten als auch die extern verlinkten "Weitere Partner") und speichert sie als CSV.

- Quelle: https://www.bde.de/verband/mitgliedschaft/foerdermitglieder/
- Ausgabe: data/bde_foerdermitglieder.csv

Hinweise:
- Einige Logos sind als reine Bilder ohne sichtbaren Text hinterlegt. In diesen Fällen werden Name/Bezeichnung über das `alt`-Attribut, `title`, `aria-label` oder notfalls den Titel der verlinkten Unterseite (H1) bestimmt.
- Doppelte Einträge (z. B. gleiche URL in mehreren Sektionen) werden dedupliziert.

In [2]:
import re
import time
import pathlib
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.bde.de/verband/mitgliedschaft/foerdermitglieder/"
OUTPUT_DIR = pathlib.Path("data")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE = OUTPUT_DIR / "bde_foerdermitglieder.csv"

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36",
}

session = requests.Session()
session.headers.update(headers)

resp = session.get(BASE_URL, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

# Helper functions

def clean_text(s: str | None) -> str:
    if not s:
        return ""
    s = re.sub(r"\s+", " ", s)
    return s.strip()


def best_label_from_anchor(a):
    # Prefer visible text
    txt = clean_text(a.get_text(" "))
    if txt:
        return txt
    # Fallback to attributes
    for attr in ("title", "aria-label"):
        v = clean_text(a.get(attr))
        if v:
            return v
    # Look for image alt inside anchor
    img = a.find("img")
    if img:
        for attr in ("alt", "title"):
            v = clean_text(img.get(attr))
            if v:
                return v
    return ""


def combine_following_content(heading_tag):
    # Combines sibling blocks until next h2/h3
    blocks = []
    for sib in heading_tag.next_siblings:
        if getattr(sib, "name", None) in ("h2", "h3"):
            break
        if getattr(sib, "name", None) in ("div", "section", "p", "ul", "ol"):
            blocks.append(sib)
    if blocks:
        return BeautifulSoup("".join(str(x) for x in blocks), "lxml")
    # Fallback to parent
    return heading_tag.parent if heading_tag.parent else soup


SOCIAL_DOMAINS = {
    "linkedin.com", "facebook.com", "twitter.com", "x.com", "youtube.com", "instagram.com",
    "xing.com", "kununu.com", "google.com", "goo.gl", "bit.ly", "t.me"
}


def is_social_url(url: str) -> bool:
    netloc = urlparse(url).netloc.lower()
    return any(dom in netloc for dom in SOCIAL_DOMAINS)


def find_external_website_on_profile(profile_url: str) -> tuple[str | None, str | None]:
    """Return (company_name, website_url) from a BDE profile page."""
    try:
        r = session.get(profile_url, timeout=30)
        if not r.ok:
            return None, None
        psoup = BeautifulSoup(r.text, "lxml")
        # Name from H1/H2
        name = None
        h = psoup.find(["h1", "h2"])  # don't require string-only
        if h:
            name = clean_text(h.get_text()) or None

        # Look for explicit external website anchors
        candidates = []
        for a in psoup.select("a[href]"):
            href = a.get("href")
            if not href:
                continue
            u = urljoin(profile_url, href)
            if "bde.de" in urlparse(u).netloc.lower():
                continue
            if is_social_url(u):
                continue
            label = best_label_from_anchor(a)
            # Score candidates: prefer those that look like a domain label (contains a dot) or have globe/WWW text
            score = 0
            if re.search(r"([a-z0-9-]+\.)+[a-z]{2,}", (label or "") + " " + u, re.I):
                score += 2
            if any(t in (label or "").lower() for t in ["www", "website", "webseite", "homepage", "zur website", "mehr infos", "informationen"]):
                score += 1
            # If a nearby section mentions "Weitere Informationen", reward similar anchors
            if psoup.find(string=re.compile("Weitere Informationen", re.I)) and re.search(r"information", (label or ""), re.I):
                score += 1
            candidates.append((score, u))
        if candidates:
            candidates.sort(key=lambda x: x[0], reverse=True)
            website = candidates[0][1]
            return name, website
        return name, None
    except Exception:
        return None, None


# Identify sections by headings
headings = soup.find_all(["h2", "h3"])  # allow nested spans, etc.
unsere_blocks = []
weitere_blocks = []
for h in headings:
    title = clean_text(h.get_text()).lower()
    if "unsere partner" in title or "unsere partner stellen sich vor" in title:
        unsere_blocks.append(combine_following_content(h))
    elif "weitere partner" in title:
        weitere_blocks.append(combine_following_content(h))

# Extract candidates
unsere_candidates = set()
weitere_candidates = set()

for blk in unsere_blocks:
    for a in blk.select("a[href]"):
        href = a.get("href") or ""
        u = urljoin(BASE_URL, href)
        parsed = urlparse(u)
        if "bde.de" in parsed.netloc.lower() and "/verband/mitgliedschaft/foerdermitglieder/" in parsed.path:
            unsere_candidates.add(u)

for blk in weitere_blocks:
    for a in blk.select("a[href]"):
        href = a.get("href") or ""
        u = urljoin(BASE_URL, href)
        parsed = urlparse(u)
        if parsed.scheme and parsed.netloc and "bde.de" not in parsed.netloc.lower():
            weitere_candidates.add(u)

# Fallbacks if nothing detected via headings
if not unsere_candidates:
    for a in soup.select('a[href*="/verband/mitgliedschaft/foerdermitglieder/"]'):
        u = urljoin(BASE_URL, a.get("href") or "")
        if u:
            unsere_candidates.add(u)

if not weitere_candidates:
    for a in soup.select("a[href]"):
        u = urljoin(BASE_URL, a.get("href") or "")
        p = urlparse(u)
        if p.scheme and p.netloc and "bde.de" not in p.netloc.lower():
            weitere_candidates.add(u)

# Build final list by visiting profile pages for unsere partner
rows = []
seen_websites = set()

for profile_url in sorted(unsere_candidates):
    name, website = find_external_website_on_profile(profile_url)
    if website and not urlparse(website).scheme:
        website = "https://" + website.lstrip("/")
    rows.append({
        "category": "unsere partner",
        "name": name,
        "website_url": website,
        "bde_profile_url": profile_url,
    })
    if website:
        seen_websites.add(website)
    time.sleep(0.2)

# Add weitere partner directly from page
for website in sorted(weitere_candidates):
    if is_social_url(website) or website in seen_websites:
        continue
    # Derive a readable name from domain as fallback
    domain = urlparse(website).netloc or website
    name_guess = re.sub(r"^www\.", "", domain)
    rows.append({
        "category": "weitere partner",
        "name": name_guess,
        "website_url": website,
        "bde_profile_url": None,
    })

# Create DataFrame
df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=["category", "name", "website_url", "bde_profile_url"])  # ensure columns

# Sort and save
df["name"] = df["name"].fillna("").apply(clean_text).replace({"": None})
df = df.sort_values(by=["category", "name", "website_url"], na_position="last").reset_index(drop=True)

df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved {len(df)} entries to {OUTPUT_FILE}")
print(df.groupby("category")["website_url"].count())

df.head(20)

Saved 55 entries to data/bde_foerdermitglieder.csv
category
unsere partner     28
weitere partner    27
Name: website_url, dtype: int64


,category,name,website_url,bde_profile_url
0,unsere partner,60 Jahre BDE – 60 Jahre Innovation,https://www.avr-rechtsanwaelte.de/,https://www.bde.de/verband/mitgliedschaft/foer...
1,unsere partner,AMCS,https://www.amcsgroup.com/de/,https://www.bde.de/verband/mitgliedschaft/foer...
2,unsere partner,APEX Container Lift Systeme GmbH – Innovation ...,https://www.apex-cls.de/,https://www.bde.de/verband/mitgliedschaft/foer...
3,unsere partner,Axians eWaste GmbH,https://www.axians-ewaste.com/,https://www.bde.de/verband/mitgliedschaft/foer...
4,unsere partner,Craemer GmbH,https://www.craemer.com/startseite,https://www.bde.de/verband/mitgliedschaft/foer...
5,unsere partner,DAKO GmbH,http://www.dako.de/entsorger,https://www.bde.de/verband/mitgliedschaft/foer...
6,unsere partner,Doppstadt Umwelttechnik GmbH,https://www.doppstadt.de/,https://www.bde.de/verband/mitgliedschaft/foer...
7,unsere partner,Dr. Ing. Wandrei GmbH,https://www.wandrei.de/,https://www.bde.de/verband/mitgliedschaft/foer...
8,unsere partner,Fagus-GreCon Greten GmbH & Co. KG,https://www.fagus-grecon.com/de/brandschutz/br...,https://www.bde.de/verband/mitgliedschaft/foer...
9,unsere partner,Feistmantl Cleaning Systems GmbH,https://feistmantl.com/,https://www.bde.de/verband/mitgliedschaft/foer...


In [3]:
df.tail(20)

,category,name,website_url,bde_profile_url
35,weitere partner,fn.legal,https://fn.legal/de/,None
36,weitere partner,gefahrgutjaeger.de,http://www.gefahrgutjaeger.de/index.htm,None
37,weitere partner,gipa.de,https://www.gipa.de/,None
38,weitere partner,loewe-container.de,https://www.loewe-container.de/,None
39,weitere partner,meiller.com,https://www.meiller.com/de/,None
40,weitere partner,mueller-umwelt.de,https://www.mueller-umwelt.de/,None
41,weitere partner,oklp.de,https://oklp.de/,None
42,weitere partner,orglmeister.de,https://www.orglmeister.de/,None
43,weitere partner,partslife.com,https://www.partslife.com/,None
44,weitere partner,pauly-rechtsanwaelte-koeln.de,https://www.pauly-rechtsanwaelte-koeln.de/,None


# Netzwerk-Seite scrapen (Kooperationen und Initiativen, Mitgliedschaften, Unsere Partner)

Diese Zelle extrahiert alle Einträge der Seite https://bde.de/verband/netzwerk/ aus den drei Slider-Sektionen:
- Kooperationen und Initiativen
- Mitgliedschaften
- Unsere Partner

Hinweis: Pro Karte gibt es i. d. R. einen externen Link zur jeweiligen Organisation. Diese werden hier direkt erfasst (internes BDE-Navigationsmaterial wird ignoriert).

In [6]:
import re
import time
import pathlib
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup
import pandas as pd

NETZWERK_URL = "https://bde.de/verband/netzwerk/"
OUT_DIR = pathlib.Path("data")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_FILE_NETZWERK = OUT_DIR / "bde_netzwerk.csv"

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36",
}

sess = requests.Session()
sess.headers.update(headers)

r = sess.get(NETZWERK_URL, timeout=30)
r.raise_for_status()
s = BeautifulSoup(r.text, "lxml")


def clean(s: str | None) -> str:
    if not s:
        return ""
    return re.sub(r"\s+", " ", s).strip()


SOCIAL = {"linkedin.com", "facebook.com", "twitter.com", "x.com", "youtube.com", "instagram.com", "xing.com"}

section_keys = [
    "kooperationen und initiativen",
    "mitgliedschaften",
    "unsere partner",
]


def is_external(u: str) -> bool:
    p = urlparse(u)
    return bool(p.scheme and p.netloc) and "bde.de" not in p.netloc.lower()


def is_social(u: str) -> bool:
    host = urlparse(u).netloc.lower()
    return any(h in host for h in SOCIAL)


def nearest_section_key(node) -> str | None:
    # Walk backwards in document order to find the nearest H2/H3 heading
    for prev in node.previous_elements:
        name = getattr(prev, "name", None)
        if name in ("h2", "h3"):
            t = clean(prev.get_text()).lower()
            for key in section_keys:
                if key in t:
                    return key
    return None


rows = []
seen = set()

for a in s.select("a[href]"):
    href = a.get("href")
    if not href:
        continue
    url = urljoin(NETZWERK_URL, href)
    if not is_external(url) or is_social(url):
        continue
    key = nearest_section_key(a)
    if key not in section_keys:
        continue
    if url in seen:
        continue
    seen.add(url)
    name = clean(a.get_text())
    if not name:
        img = a.find("img")
        if img and img.get("alt"):
            name = clean(img.get("alt"))
    if not name:
        # fallback from domain
        domain = urlparse(url).netloc or url
        name = re.sub(r"^www\.", "", domain)
    rows.append({
        "section": key,
        "name": name or None,
        "url": url,
    })

net_df = pd.DataFrame(rows)
if net_df.empty:
    net_df = pd.DataFrame(columns=["section", "name", "url"])  # ensure columns
else:
    net_df = net_df.sort_values(["section", "name", "url"], na_position="last").reset_index(drop=True)

net_df.to_csv(OUT_FILE_NETZWERK, index=False)
print(f"Saved {len(net_df)} entries to {OUT_FILE_NETZWERK}")
print(net_df["section"].value_counts(dropna=False))
net_df.head(20)

Saved 61 entries to data/bde_netzwerk.csv
section
unsere partner                   47
kooperationen und initiativen    10
mitgliedschaften                  4
Name: count, dtype: int64


,section,name,url
0,kooperationen und initiativen,Aktion Biotonne Deutschland,https://aktion-biotonne-deutschland.de/
1,kooperationen und initiativen,BAV – Bundesverband der Altholzaufbereiter und...,https://altholzverband.de
2,kooperationen und initiativen,Bundesverband Altöl e. V.,https://bva-altoelrecycling.de/
3,kooperationen und initiativen,Bundesvereinigung Recycling-Baustoffe e. V. (BRB),https://recyclingbaustoffe.de/
4,kooperationen und initiativen,Germany Trade and Invest,https://www.gtai.de/gtai-de
5,kooperationen und initiativen,Gesamtverband Schadstoffsanierung (GVSS) e. V.,https://www.gesamtverband-schadstoff.de/
6,kooperationen und initiativen,"Initiative ""Mülltrennung wirkt!""",https://www.muelltrennung-wirkt.de/
7,kooperationen und initiativen,Interessengemeinschaft der Aufbereiter und Ver...,https://igam-hmva.de/
8,kooperationen und initiativen,PREVENT Abfall-Allianz,https://prevent-waste.net/
9,kooperationen und initiativen,"ZER-QMS Zertifizierungsstelle, Qualitäts- und ...",https://www.zer-qms.de/


In [7]:
# Vorschau Netzwerk mit klickbaren Links
import pandas as pd
from IPython.display import HTML

net_df = pd.read_csv("data/bde_netzwerk.csv")

def make_anchor(name, url):
    if not isinstance(url, str) or not url:
        return ""
    label = name.strip() if isinstance(name, str) and name.strip() else url
    return f'<a href="{url}" target="_blank" rel="noopener noreferrer">{label}</a>'

net_df["link"] = [make_anchor(n, u) for n, u in zip(net_df.get("name"), net_df.get("url"))]
HTML(net_df[["section", "name", "url", "link"]].to_html(escape=False, index=False))

section,name,url,link
kooperationen und initiativen,Aktion Biotonne Deutschland,https://aktion-biotonne-deutschland.de/,Aktion Biotonne Deutschland
kooperationen und initiativen,BAV – Bundesverband der Altholzaufbereiter und -verwerter e. V.,https://altholzverband.de,BAV – Bundesverband der Altholzaufbereiter und -verwerter e. V.
kooperationen und initiativen,Bundesverband Altöl e. V.,https://bva-altoelrecycling.de/,Bundesverband Altöl e. V.
kooperationen und initiativen,Bundesvereinigung Recycling-Baustoffe e. V. (BRB),https://recyclingbaustoffe.de/,Bundesvereinigung Recycling-Baustoffe e. V. (BRB)
kooperationen und initiativen,Germany Trade and Invest,https://www.gtai.de/gtai-de,Germany Trade and Invest
kooperationen und initiativen,Gesamtverband Schadstoffsanierung (GVSS) e. V.,https://www.gesamtverband-schadstoff.de/,Gesamtverband Schadstoffsanierung (GVSS) e. V.
kooperationen und initiativen,"Initiative ""Mülltrennung wirkt!""",https://www.muelltrennung-wirkt.de/,"Initiative ""Mülltrennung wirkt!"""
kooperationen und initiativen,Interessengemeinschaft der Aufbereiter und Verwerter von Müllverbrennungsschlacken (IGAM),https://igam-hmva.de/,Interessengemeinschaft der Aufbereiter und Verwerter von Müllverbrennungsschlacken (IGAM)
kooperationen und initiativen,PREVENT Abfall-Allianz,https://prevent-waste.net/,PREVENT Abfall-Allianz
kooperationen und initiativen,"ZER-QMS Zertifizierungsstelle, Qualitäts- und Umweltgutachter GmbH",https://www.zer-qms.de/,"ZER-QMS Zertifizierungsstelle, Qualitäts- und Umweltgutachter GmbH"


In [8]:
# Kombiniere alle Daten in ein gemeinsames CSV
import pandas as pd
from pathlib import Path

out_dir = Path("data")
combined_path = out_dir / "bde_scrape.csv"

# Load Fördermitglieder (if exists)
df_foerd = None
p1 = out_dir / "bde_foerdermitglieder.csv"
if p1.exists():
    df_foerd = pd.read_csv(p1)
    # Normalize column names
    df_foerd = df_foerd.rename(columns={
        "category": "section",
        "website_url": "url",
        "bde_profile_url": "profile_url",
    })
    if "section" not in df_foerd.columns:
        df_foerd["section"] = "foerdermitglieder"
    df_foerd["source_page"] = "foerdermitglieder"

# Load Netzwerk (if exists)
df_netz = None
p2 = out_dir / "bde_netzwerk.csv"
if p2.exists():
    df_netz = pd.read_csv(p2)
    df_netz["profile_url"] = None
    df_netz["source_page"] = "netzwerk"

# Concatenate
frames = [d for d in [df_foerd, df_netz] if d is not None]
if frames:
    df_all = pd.concat(frames, ignore_index=True, sort=False)
else:
    df_all = pd.DataFrame(columns=["section", "name", "url", "profile_url", "source_page"])  # empty

# Standardize columns and order
for c in ["section", "name", "url", "profile_url", "source_page"]:
    if c not in df_all.columns:
        df_all[c] = None

# Deduplicate by URL + name to be safe
df_all = df_all.drop_duplicates(subset=["name", "url"]).sort_values(
    by=["source_page", "section", "name", "url"], na_position="last"
).reset_index(drop=True)

# Save
df_all.to_csv(combined_path, index=False)
print(f"Saved {len(df_all)} combined rows to {combined_path}")
try:
    print(df_all.groupby(["source_page", "section"]).size())
except Exception:
    pass

df_all.head(20)

Saved 108 combined rows to data/bde_scrape.csv
source_page        section                      
foerdermitglieder  unsere partner                   28
                   weitere partner                  27
netzwerk           kooperationen und initiativen    10
                   mitgliedschaften                  4
                   unsere partner                   39
dtype: int64


,section,name,url,profile_url,source_page
0,unsere partner,60 Jahre BDE – 60 Jahre Innovation,https://www.avr-rechtsanwaelte.de/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
1,unsere partner,AMCS,https://www.amcsgroup.com/de/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
2,unsere partner,APEX Container Lift Systeme GmbH – Innovation ...,https://www.apex-cls.de/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
3,unsere partner,Axians eWaste GmbH,https://www.axians-ewaste.com/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
4,unsere partner,Craemer GmbH,https://www.craemer.com/startseite,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
5,unsere partner,DAKO GmbH,http://www.dako.de/entsorger,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
6,unsere partner,Doppstadt Umwelttechnik GmbH,https://www.doppstadt.de/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
7,unsere partner,Dr. Ing. Wandrei GmbH,https://www.wandrei.de/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
8,unsere partner,Fagus-GreCon Greten GmbH & Co. KG,https://www.fagus-grecon.com/de/brandschutz/br...,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
9,unsere partner,Feistmantl Cleaning Systems GmbH,https://feistmantl.com/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder


In [11]:
df_all.sample(10)

,section,name,url,profile_url,source_page
26,unsere partner,ZER-QMS Zertifizierungsstelle GmbH,https://www.zer-qms.de,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
100,unsere partner,Terberg HS GmbH,https://www.terberg-hs.de/,None,netzwerk
48,weitere partner,satelliteindustries.de,https://www.satelliteindustries.de/,NaN,foerdermitglieder
34,weitere partner,faun.com,https://www.faun.com/,NaN,foerdermitglieder
79,unsere partner,Feistmantl Cleaning Systems GmbH,http://www.feistmantl.com/,None,netzwerk
91,unsere partner,PRESTO GmbH & Co. KG,http://www.presto.eu/de,None,netzwerk
19,unsere partner,PROLOGA,https://prologa-group.com/,https://www.bde.de/verband/mitgliedschaft/foer...,foerdermitglieder
97,unsere partner,Satellite Industries GmbH,http://www.satelliteindustries.de/,None,netzwerk
96,unsere partner,SULO Deutschland GmbH,https://sulo.de/,None,netzwerk
68,mitgliedschaften,Landesvereinigung der Unternehmensverbände Nor...,https://www.unternehmer.nrw/,None,netzwerk
